# Recommendation
Author: Ruben Schild<br>
Student number: 650580<br>
Date: 31-5-2022

For this notebook to work you must have installed the following packages (usually via pip install *packageName*:
* numpy
* pandas
* **\<add other packages\>**

From these we will need the following libraries:

In [1]:
# pandas as pd
try:
    import pandas as pd
    print('Pandas already installed, only imported')
except:
    !pip install pandas
    import pandas as pd
    print('Pandas was not installed, installed and imported')
    
# numpy as np
try:
    import numpy as np
    print('NumPy already installed, only imported')
except:
    !pip install numpy
    import numpy as np
    print('NumPy was not installed, installed and imported')
    
# pyplot as plt
try:
    import matplotlib.pyplot as plt
    print('PyPlot already installed, only imported')
except:
    !pip install matplotlib
    import matplotlib.pyplot as plt
    print('PyPlot was not installed, installed and imported')    

Pandas already installed, only imported
NumPy already installed, only imported
PyPlot already installed, only imported


# 1. The Item-Based Recommender

We are going to build an item-based recommender for **<span style='background:yellow'>\<YOUR INPUT></span>**. In this recommender we could for example **<span style='background:yellow'>\<give a brief description of an example how your recommender would work></span>**.

To begin we'll need to load some data.

In [5]:
# enter code to load your example data file(s)
books_df = pd.read_csv('../data/books.csv', on_bad_lines='skip', sep=';', encoding='latin-1')


C:\Users\ruben\AppData\Local\Temp\ipykernel_14432\3480200639.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('../data/books.csv', on_bad_lines='skip', sep=';', encoding='latin-1')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


**<span style='background:yellow'>\<explain a bit about the data file(s), if you have multiple and need to merge them, use the code cell below></span>**

In [3]:
# Optional depending on your dataset, enter code to select appropriate columns, and merge the files.


We need one row per user, and his/her scores for each **<span style='background:yellow'>\<YOUR OBJECT></span>**. We can do this by creating a pivot table.

In [4]:
# code to create pivot table


We also need to take the counts into consideration so lets keep those into their own dataframe, together with the average score:

In [5]:
# code to make a dataframe with the unique objects, their average score and the number of scores


To decide on a cut-off point as minimum number of scores to be eligable we can plot the number of scores in a histogram:

In [6]:
# code to generate the histogram


From the histogram we can see that **<span style='background:yellow'>\<Explain in your own words what you notice and what you decide on as the cut-off point, explain why></span>**

Finally we can create the code to perform a recommendation. First we can select one particular **<span style='background:yellow'>\<YOUR OBJECT></span>**

In [7]:
# code to select one field


And then to find the recommendation based on the correlations.

In [8]:
# code to create the recommendation


**<span style='background:yellow'>\<interpret the result></span>**

It might be nice to have one simple function that could do all the work for us. It should have as input parameter a field and then returns the recommendation:

In [9]:
# code for a python function



Lets test the function:

In [10]:
# code to test the function


<font size="5"><center>-----Chapters 1 is required to be fully completed to get a 60, the next few chapters will give a +10 for each chapter.<br> 
    However no template is available for these chapters. You will have to create it yourself.
    ----</center></font>

# 2. Negative correlations

The recommenders so far used, are looking at what if I give a high rating for this, what do you then recommend me. But how about if I give a low rating for something, what would you then recommend me?

# 3. Content Based

Add a chapter showing how a basic content based recommender might work. A starting point for this might be https://www.datacamp.com/community/tutorials/recommender-systems-python (at about 1/3 of the page it starts explaining this in the section called ‘Content based’.

# 4. User Based

Add a chapter showing how a basic user based recommender might work. A starting point for this might be https://realpython.com/build-recommendation-engine-collaborative-filtering/

# 5. Combination

Combine two or more recommendation techniques to get an even better recommendation